# Simulating Shoreline Change using Coupled Coastsat and Coastline Evolution Model (CEM) - Part 2


#### In this lab users will extract a satellite derived shoreline in a location of their choosing. They will then evolve this shoreline using the Coastal Evolution model. They will be able to tune wave characteristics and explore the role of dominant wave climate in producing longterm change of an area relevant to their interests. 

#### Part 1 of the lab extracts all historical observations and prepares them for input into the CEM
#### Part 2 of the lab uses the CEM to evolve the extracted shoreline

Important references:

1. Ashton, A.D., Murray, B., Arnault, O. 2001. Formation of coastline features by large-scale instabilities induced by high-angle waves, Nature 414.
2. Ashton A.D., Murray A.B. [High-Angle Wave Instability and Emergent Shoreline Shapes: 1. Wave Climate Analysis and Comparisons to Nature.](https://agupubs.onlinelibrary.wiley.com/doi/full/10.1029/2005JF000422) Journal of Geophysical Research. Volume 111. 15 December 2006. <br>
3. Ashton A.D., Murray A.B. [High-Angle Wave Instability and Emergent Shoreline Shapes: 2. Wave Climate Analysis and Comparisons to Nature.](https://agupubs.onlinelibrary.wiley.com/doi/full/10.1029/2005JF000423) Journal of Geophysical Research. Volume 111. 15 December 2006.
4. Vos K., Splinter K.D., Harley M.D., Simmons J.A., Turner I.L. (2019). CoastSat: a Google Earth Engine-enabled Python toolkit to extract shorelines from publicly available satellite imagery. Environmental Modelling and Software. 122, 104528. https://doi.org/10.1016/j.envsoft.2019.104528 (Open Access)


### 1. Import Relevant Libraries

We will be using some default Python libraries, as well as importing some functions we wrote ourselves!

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import path
%matplotlib inline
import pandas as pd
from scipy import stats,signal
from IPython.display import clear_output

from pymt.models import Cem, Waves
import buoypy as bp
import data_prep_fun as dpf
import runmodels_functions as fun

### 2. Load Shoreline Data from CoastSAT & NDBC Buoy Wave Data

These workflows are derived from the "Extract Wave Data" & "Convert CoastSat shoreline to grid for CEM" sections in the CoastSat & Wave Data notebook. Please see that file for more information on these processes.

First, let's get our wave data:

In [ ]:
#From previous nb
Hs, Tp, Dir = 0.486, 3.156, 214.

We will be using the significant wave height, wave period, and wave direction data from the NDBC buoy as inputs into the WAVES model. Now let's look at our shoreline:

In [ ]:
# dummy shoreline that will be replaced by CoastSat shoreline
xy = np.loadtxt('Erie_spit_ESPIn.xy')
# shoreline
x=xy[:,0]
y=xy[:,1]
dx=100
dy=100

dpf.shorelinetogrid(x,y,dx,dy,plotdata=True);

We have to adjust some aspects of the elevation data to get the correct orientation and make it run more smoothly within CEM. Below we rotate the shoreline to be compatible with the CEM boundary conditions.

In [ ]:
a = -25*np.pi/180
x = (x ) * np.cos(a) - (y ) * np.sin(a)  
y = (x ) * np.sin(a) + (y ) * np.cos(a)  

We must also make the shore/shelf gradients smoother for CEM to calculate the shore/self slopes.
This is done by the 2d convolution below. Some additional cleaning is also needed to get the initial domain in recognizable form...

In [ ]:
[xg,yg,elev] = dpf.shorelinetogrid(x,y,dx,dy,plotdata=True);
plt.close()
z_elev = (elev.T)

z_elev[z_elev==-5] = -1
domain = z_elev*-1 

##This can be finicky.
domain = signal.convolve2d(domain,np.ones([10,10])/(10**2),mode='same',boundary='symm')
domain[z_elev==-1] = 1
domain[:47,:] = 1

Let's visualize this updated coastline data with the function `plot_coast`. This will confirm that we have effectively translated the spit into model space.

In [ ]:
fun.plot_coast(domain,dx,dy)

Looks pretty similar! Last step is to find the average shoreface and shelf slope. We will do this by appoximating a straight transect that extends from the beach off shore:

In [ ]:
profile = np.copy(domain)[70:,80]
x = np.arange(len(profile))*dx/1000
plt.plot(x,profile/1000,'-o',linewidth=5,alpha=0.6,label='Profile')

plt.plot(x,-.005*x+.0035,'--k',label='Shelf Slope ~ 0.005',alpha=0.8,linewidth=2)


plt.legend(fontsize=12)
plt.ylabel('Depth (km)',fontsize=20)
plt.xlabel('Crosshsore (km)',fontsize=20)


In [ ]:
###Looks to be about
shelf_slope = 0.005
##guessing the shoreface params
shoreface_slope = 0.01
shoreface_depth = 15 #meters

Now that we have processed the shoreline, we can initialize our models.

### 3. Model Setup
We are using the [Coastal Evolution Model (CEM)](https://csdms.colorado.edu/wiki/Model_help:CEM) and the [WAVES](https://csdms.colorado.edu/wiki/Model_help:Waves) models. The CEM simulates morphodynamic evolution of coastlines under varying wave climates. WAVES is the model that calculates the input wave energetics used in CEM as a function of wave period (T), deep water wave height (H<sub>0</sub>), and wave angle criteria. Run the help() commands on each function and read through their documentation.

In [ ]:
cem = Cem()
waves = Waves()

In [ ]:
help(cem)

In [ ]:
help(waves)

Let's look at what variables go into and come out of each model. We'll first look at ***WAVES***:

In [ ]:
waves.input_var_names

### WAVES Model Input Parameters
- `sea_surface_water_wave__height` --> Deep water wave height (H<sub>0</sub>)
- `sea_surface_water_wave__period` --> Wave period (T)
- `sea_shoreline_wave~incoming~deepwater__ashton_et_al_approach_angle_highness_parameter` --> Proportion of high angle waves (U)
> "The variable U controls the general directional spread of the approaching waves, here split into whether waves approach from angles great than or less than the one which maximized alongshore sediment transport (approximately 45 degrees)...U varies between 0-1, controls the fraction of high-angle waves, where a value of less than 0.5 indicates wave energy predominately approaching from a low angle and a designation greater than 0.5 indicates a predominance of high-angle (unstable waves)."
> "This variable is probably the most important control on the behavior of coupled CEM-WAVES simulations using this simplified wave climate scheme. If CEM is being employed to simulate the self-organization of a coast, values of U larger than 0.5 should be used. For scenarios involving delta evolution, values less than 0.5 tend to be more reasonable (unless a local wave climate has that character). There is no specific method for deriving these terms (U and A) from natural wave climate conditions. Either A and U can be toggled to emulate a natural wave or other methods can be employed (not yet functional) where a natural wave climate can be enetered into the model." 
- `sea_shoreline_wave~incoming~deepwater__ashton_et_al_approach_angle_asymmetry_parameter` --> Asymmetry of wave direction (A)
> "The variable A controls the directional distribution of waves, representing the fraction of waves approaching from the left or right, from the perspective of a person looking out to sea. It is designated to be a value between 0 and 1. A designation of greater than 0.5 indicates that the majority of wave energy is approaching from the left where a designation of 1.0 indicates all wave energy approaches from the left. A designation of 0.5 indicates wave energy approach is evenly distributed between the left and right. A designation of less than 0.5 indicates the majority of wave energy is approaching from the right where a designation of 0 indicates all wave energy approaches from the right."

[WAVES Documentation](https://csdms.colorado.edu/wiki/Model_help:Waves)

In [ ]:
waves.output_var_names

### WAVES Model Output Parameters
- `sea_surface_water_wave__min_of_increment_of_azimuth_angle_of_opposite_of_phase_velocity` --> Min wave refraction angle ($\alpha$)
- `sea_surface_water_wave__azimuth_angle_of_opposite_of_phase_velocity` --> Wave refraction angle ($\alpha$)
- `sea_surface_water_wave__mean_of_increment_of_azimuth_angle_of_opposite_of_phase_velocity` --> Mean wave refraction angle ($\alpha$)
- `sea_surface_water_wave__max_of_increment_of_azimuth_angle_of_opposite_of_phase_velocity` --> Max wave refraction angle ($\alpha$)
- `sea_surface_water_wave__height` --> Wave period (T)
- `sea_surface_water_wave__period` --> Deep water wave height (H<sub>0</sub>)

[WAVES Documentation](https://csdms.colorado.edu/wiki/Model_help:Waves)

Now, let's look at ***CEM***:

In [ ]:
cem.input_var_names

### CEM Input Parameters
- `sea_surface_water_wave__azimuth_angle_of_opposite_of_phase_velocity` --> Wave refraction angle ($\alpha$)
- `land_surface_water_sediment~bedload__mass_flow_rate` --> Riverine sediment load (Q<sub>s</sub>)
- `sea_surface_water_wave__period` --> Wave period (T)
- `sea_surface_water_wave__height` --> Wave height (H)
- `land_surface__elevation` --> Land elevation (L)
- `model__time_step` --> Time step for model runs (dt)

[CEM Documentation](https://csdms.colorado.edu/wiki/Model_help:CEM)

In [ ]:
cem.output_var_names

### CEM Output Parameters
- `basin_outlet~coastal_center__x_coordinate` --> Riverine outlet longitude
- `basin_outlet~coastal_water_sediment~bedload__mass_flow_rate` --> Coastal sediment load (Q<sub>s c</sub>)
- `land_surface__elevation` --> Land elevation (L)
- `sea_water__depth` --> Water depth (Z)
- `basin_outlet~coastal_center__y_coordinate` --> Riverine outlet latitude
- `model__time_step` --> Time step for model runs (dt)

[CEM Documentation](https://csdms.colorado.edu/wiki/Model_help:CEM)

Below we reassign the long variable names to simple characters. Saves us typing in the future! See above for documentation on what each variable represents.

In [ ]:
H0 = 'sea_surface_water_wave__height'
T = 'sea_surface_water_wave__period'
U = 'sea_shoreline_wave~incoming~deepwater__ashton_et_al_approach_angle_highness_parameter'
A = 'sea_shoreline_wave~incoming~deepwater__ashton_et_al_approach_angle_asymmetry_parameter'
alpha = 'sea_surface_water_wave__azimuth_angle_of_opposite_of_phase_velocity'
Qs = 'land_surface_water_sediment~bedload__mass_flow_rate'
Z = 'sea_water__depth'
L = 'land_surface__elevation'
dt = 'model__time_step'

### 4. Initialize Models with Input Criteria
Here we are creating a _dictionary_ to assign value to our various input parameters for both the CEM and WAVES models. Defining CEM's parameter values this way facilitates transparency and easy manipulation of the variables. This dictionary will then be passed into the **initialize_models** function which sets up our models.

For our wave parameters, we are using the data from the "Extract Wave Data" section of the CoastSat notebook. Please see that file for more information on the data extraction.

*To convert wave direction to the **A** parameter, we use the following formula:*

$$\frac{|Dir-270|}{180}$$

*where **Dir** represents wave direction.*

In [ ]:
#convert wave direction to asymmetry term
Ap = 1-float(abs(Dir-270)/180)

## We are using 1-Ap because the waves in our domain are coming from the left.

In [ ]:
params = {
    
    ## CEM 
    'grid_spacing'    : dx,                #meters
    'shelf_slope'     : shelf_slope,
    'shoreface_depth' : shoreface_depth,   #meters
    'shoreface_slope' : shoreface_slope,
    'model__time_step': 0.1,               #years
    
    ##WAVES
    'sea_surface_water_wave__height' : Hs, #meters
    'sea_surface_water_wave__period' : Tp, #seconds
    'sea_shoreline_wave~incoming~deepwater__ashton_et_al_approach_angle_highness_parameter': 0.6,
    'sea_shoreline_wave~incoming~deepwater__ashton_et_al_approach_angle_asymmetry_parameter': Ap
    
}

In the CEM, cells can either be water or land. Land cells will have a uniform height that is greater than the water depth. They will only be affected by the coastal processes of _alongshore sediment transport_ and _wave action. Inland cells will not be affected. Water cells will have a set bathymetry that will respond to sediment supply and wave action. The model calculates the bathymetry through the inner shelf slope (`shelf_slope`), depth at which the shoreface ends (`shoreface_depth`), and the shoreface slope (`shoreface_slope`). We can take a look at an example [here](Example_run___SPITS.ipynb) showing how the CEM runs without an input shoreline.

We initialize the model with the `initialize_models` function.

In [ ]:
fun.initialize_models(params,domain,cem,waves)

Let's look at our input coastline again with the **plot_coast** function as our baseline. But now we use the BMI commant below to see if the CEM took our input and didnt do anything too crazy when it recalculates the slopes and depths.

We want to look at the input variable 'land_surface__elevation' but this value is transformed into a 1d array in the model so we want to reshape it into the 2d array like our initial domain:

In [ ]:
land = cem.get_value('land_surface__elevation')
land = land.reshape(  domain.shape  )

fun.plot_coast(land,dx,dy)

LOOKS KILLER!!! If you get any weirdness here go back up and smooth (convolve) the domain a bit more and try again.

The `run_model_loop function` will run the CEM and can display an animation of the evolving shoreline. The first input expresses the model run time (in years). The function displays an animation by default but you can set `animate=False` to decrease model computation time.

In [ ]:
##RUN TILL 2021
T = 36 #years
fun.run_model_loop(T,domain,cem,waves,update_ani_years=1)

Let's look at a raster of change to show how the sediment moved throughout the model run. Areas of deposition are visualized in yellow, and erosion is in dark blue.

In [ ]:
final = cem.get_value('land_surface__elevation').reshape(domain.shape)
fun.plot_coast(final-land,dx,dy)

Now, just for fun, lets run this a thousand years into the future!

In [ ]:
##RUN INTO THE FUTURE!!!!!
T = 1000 #years
fun.run_model_loop(T,domain,cem,waves,update_ani_years=10)

Congratulations! We just ran the CEM on a real coastline input. Let's compare our results to see how our models works with the CoastSAT data. Here's what the Lake Erie Spit looked like in 2020:

<img src="Presque_Ilse_State_Park_9_10_2020.png" alt="Presque Ilse State Park" width="500"/>

And here is our CEM result:

In [ ]:
fun.plot_coast(final,dx,dy)

Looks pretty close! Models rarely fully replicate all changes that happen in a real system due to various assumptions and important forcings neglected (i.e., storm events). Nevertheless, CEM does a pretty good job at simulating coastline change on a real spit.